## Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as acc
# from mlxtend.feature_selection import SequentialFeatureSelector as sfs
import time
import calendar

from collections import Counter #Contador das classes
from imblearn.under_sampling import RandomUnderSampler #UnderSampler

## Carregando o dataset

In [ ]:
pathDadosAbertos = "/content/drive/MyDrive/doutorado/Outros/Projeto Arboviroses/baseZika/zika/csv/" #@param

In [ ]:
# Array que terá o caminho/url de todos os datasets
dataset_paths = []

# TODO: adicionar os csvs de Recife (para ler eles, talvez seja preciso adicionar encoding="ISO-8859-1" no pd.read_csv)
dataset_paths.extend([
  f'{pathDadosAbertos}ZIKABR16.csv',
  f'{pathDadosAbertos}ZIKABR17.csv',
  f'{pathDadosAbertos}ZIKABR18.csv',
  f'{pathDadosAbertos}ZIKABR19.csv',
  f'{pathDadosAbertos}ZIKABR20.csv',
  f'{pathDadosAbertos}ZIKABR21.csv'
])



## Unificando a base

In [ ]:
all_datasets = []
for path in dataset_paths:
  dt = pd.read_csv(path, sep=',')
  print(f'Loading {path} - Shape {dt.shape}')
  all_datasets.append(dt)

base = pd.concat(all_datasets, join='inner')
print(f'Rows (inner): {base.shape[0]}')
print(f'Columns (inner): {base.shape[1]}')
base.shape

Loading /content/drive/MyDrive/doutorado/Outros/Projeto Arboviroses/baseZika/zika/csv/ZIKABR16.csv - Shape (281464, 38)
Loading /content/drive/MyDrive/doutorado/Outros/Projeto Arboviroses/baseZika/zika/csv/ZIKABR17.csv - Shape (32684, 38)
Loading /content/drive/MyDrive/doutorado/Outros/Projeto Arboviroses/baseZika/zika/csv/ZIKABR18.csv - Shape (20584, 38)
Loading /content/drive/MyDrive/doutorado/Outros/Projeto Arboviroses/baseZika/zika/csv/ZIKABR19.csv - Shape (30500, 38)
Loading /content/drive/MyDrive/doutorado/Outros/Projeto Arboviroses/baseZika/zika/csv/ZIKABR20.csv - Shape (20867, 38)
Loading /content/drive/MyDrive/doutorado/Outros/Projeto Arboviroses/baseZika/zika/csv/ZIKABR21.csv - Shape (18680, 38)
Rows (inner): 404779
Columns (inner): 38


(404779, 38)

In [ ]:
base.columns

Index(['TP_NOT', 'ID_AGRAVO', 'CS_SUSPEIT', 'DT_NOTIFIC', 'SEM_NOT', 'NU_ANO',
       'SG_UF_NOT', 'ID_MUNICIP', 'ID_REGIONA', 'DT_SIN_PRI', 'SEM_PRI',
       'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N', 'SG_UF',
       'ID_MN_RESI', 'ID_RG_RESI', 'ID_PAIS', 'NDUPLIC_N', 'IN_VINCULA',
       'DT_INVEST', 'ID_OCUPA_N', 'CLASSI_FIN', 'CRITERIO', 'TPAUTOCTO',
       'COUFINF', 'COPAISINF', 'COMUNINF', 'DOENCA_TRA', 'EVOLUCAO',
       'DT_OBITO', 'DT_ENCERRA', 'CS_FLXRET', 'FLXRECEBI', 'TP_SISTEMA',
       'TPUNINOT'],
      dtype='object')

## Salvando a base unificada

In [ ]:
base

,TP_NOT,ID_AGRAVO,CS_SUSPEIT,DT_NOTIFIC,SEM_NOT,NU_ANO,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,DT_SIN_PRI,...,COPAISINF,COMUNINF,DOENCA_TRA,EVOLUCAO,DT_OBITO,DT_ENCERRA,CS_FLXRET,FLXRECEBI,TP_SISTEMA,TPUNINOT
0,2,A928,NaN,2016-01-01,201552,2016,17,170230,NaN,2016-01-01,...,0,NaN,NaN,NaN,NaN,2016-03-07,0,2,1,NaN
1,2,A928,NaN,2016-01-01,201552,2016,29,292040,1385.0,2015-12-29,...,1,292040.0,2.0,1.0,NaN,2016-06-02,0,2,1,NaN
2,2,A928,NaN,2016-01-01,201552,2016,29,292040,1385.0,2016-01-01,...,1,292040.0,2.0,1.0,NaN,2016-06-02,0,2,1,NaN
3,2,A928,NaN,2016-01-01,201552,2016,29,293240,1394.0,2015-12-28,...,1,293240.0,2.0,1.0,NaN,2016-01-02,0,2,1,NaN
4,2,A928,NaN,2016-01-01,201552,2016,32,320530,1510.0,2015-12-31,...,1,320530.0,2.0,1.0,NaN,2016-01-25,0,2,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18675,2,A928,NaN,2021-12-31,202152,2021,50,500295,1975.0,2021-12-29,...,0,NaN,NaN,NaN,NaN,NaN,0,2,1,NaN
18676,2,A928,NaN,2021-12-31,202152,2021,52,520870,1779.0,2021-12-30,...,0,NaN,NaN,NaN,NaN,NaN,0,2,1,NaN
18677,2,A928,NaN,2021-12-31,202152,2021,26,261110,1503.0,2021-12-29,...,0,NaN,NaN,NaN,NaN,NaN,0,2,1,NaN
18678,2,A928,NaN,2021-12-31,202152,2021,11,110025,1474.0,2021-12-27,...,0,NaN,NaN,NaN,NaN,NaN,0,2,1,NaN


In [ ]:
path_destino = '/content/drive/MyDrive/doutorado/Outros/Projeto Arboviroses/datasets/' #@param
base.to_csv(path_destino+'dataset_unificado.csv', sep=',', index=False)